# routes

>  Route handlers for the contacts application

In [ ]:
#| default_exp routes

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fasthtml.common import *
from todo.db import *
from todo.core import *

## Helper Functions

In [ ]:
#| export

def todo_routes(rt):
    "Define todo management routes"
    
    @rt("/")
    def get(auth):
        title = f"{auth}'s Todo list"
        top = Grid(H1(title), Div(A('logout', href='/logout'), style='text-align: right'))
        new_inp = Input(id="new-title", name="title", placeholder="New Todo")
        add = Form(Group(new_inp, Button("Add")),
                  hx_post="/", target_id='todo-list', hx_swap="afterbegin")
        frm = Form(*todos(order_by='priority'),
                  id='todo-list', cls='sortable', hx_post="/reorder", hx_trigger="end")
        card = Card(Ul(frm), header=add, footer=Div(id='current-todo'))
        return Title(title), Container(top, card)
    
    @rt("/reorder")
    def post(id:list[int]):
        for i,id_ in enumerate(id): todos.update({'priority':i}, id_)
        return tuple(todos(order_by='priority'))
    
    @rt("/todos/{id}")
    def delete(id:int):
        todos.delete(id)
        return clr_details()
    
    @rt("/edit/{id}")
    def get(id:int):
        res = Form(Group(Input(id="title"), Button("Save")),
            Hidden(id="id"), CheckboxX(id="done", label='Done'),
            Textarea(id="details", name="details", rows=10),
            hx_put="/", target_id=f'todo-{id}', id="edit")
        return fill_form(res, todos[id])
    
    @rt("/")
    def put(todo: Todo):
        return todos.update(todo), clr_details()
    
    @rt("/")
    def post(todo:Todo):
        new_inp = Input(id="new-title", name="title", placeholder="New Todo", hx_swap_oob='true')
        return todos.insert(todo), new_inp
    
    @rt("/todos/{id}")
    def get(id:int):
        todo = todos[id]
        btn = Button('delete', hx_delete=f'/todos/{todo.id}',
                    target_id=f'todo-{todo.id}', hx_swap="outerHTML")
        return Div(H2(todo.title), Div(todo.details, cls="markdown"), btn)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()